## pytesseract 설정

In [2]:
import pytesseract
from PIL import Image
import pandas as pd
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import cv2
import os
from hanspell import spell_checker

In [3]:
pytesseract.pytesseract.tesseract_cmd = R'C:\Program Files\Tesseract-OCR\tesseract'

## 마우스로 크롭

In [10]:
import cv2
 
isDragging = False
x0, y0, w, h = -1, -1, -1, -1
blue, red = (255, 0, 0), (0, 0, 255)
 
def onMouse(event, x, y, flags, param):
    global isDragging, x0, y0, img
    if event == cv2.EVENT_LBUTTONDOWN:
        isDragging = True
        x0 = x
        y0 = y
    elif event == cv2.EVENT_MOUSEMOVE:
        if isDragging:
            img_draw = img.copy()
            cv2.rectangle(img_draw, (x0, y0), (x, y), blue, 2)
            cv2.imshow('img', img_draw)
            return img_draw
    elif event == cv2.EVENT_LBUTTONUP:
        if isDragging:
            isDragging = False
            w = x - x0
            h = y - y0
            if w > 0 and h > 0:
                img_draw = img.copy()
                cv2.rectangle(img_draw, (x0, y0), (x, y), red, 2)
                cv2.imshow('img', img_draw)
                roi = img[y0:y0+h, x0:x0+w]
                cv2.imshow('cropped', roi)
                cv2.moveWindow('cropped', 0, 0)
                cv2.imwrite('./cropped.png', roi)
            else:
                cv2.imshow('img', img)
                print('drag should start from left-top side')

In [16]:
img = cv2.imread('./img/1004.jpg')

cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)
cv2.waitKey()
cv2.destroyAllWindows()

drag should start from left-top side
drag should start from left-top side


In [17]:
img_cropped=Image.open('./cropped.png')
img_cropped.show()

In [18]:
pytesseract.image_to_string(img_cropped,lang='kor')

'데누뉴(데두:외국신/미국,브라일,파라과이흥).두\n두유(대두:외국산)정제소금].정제수.발효식:\n\n정제소금(국산)변성전분,        출물레온착좁액:\n\n산잔탄검,효모추출물 혼합제제(시클로덱:\n\n엔-록1930 .트리아세틴안나토색소,   .유화제)이디. 타 에|\n\n이칼슴이나! 륨(산화방지제) 치자황색소미빼26343\n\n모딸0\n때>써0918\n\n'

## 데이터에 적용

### 원료

In [72]:
path='./img/기준폴더/원료/'
idx_lst=os.listdir(path)
idx_lst

['1010.jpg',
 '1012.jpg',
 '1042.jpg',
 '1057.jpg',
 '1073.jpg',
 '1089.jpg',
 '1133.jpg',
 '1144.jpg',
 '1148.jpg',
 '115.jpg',
 '1151.jpg',
 '1163.jpg',
 '1167.jpg',
 '1198.jpg',
 '1199.jpg',
 '1206.jpg',
 '1209.jpg',
 '1250.jpg',
 '1273.jpg',
 '1286.jpg',
 '1293.jpg',
 '1302.jpg',
 '1335.jpg',
 '1340.jpg',
 '1349.jpg',
 '1378.jpg',
 '1399.jpg',
 '14.jpg',
 '1407.jpg',
 '1408.jpg',
 '1409.jpg',
 '1430.jpg',
 '1436.jpg',
 '1437.jpg',
 '1489.jpg',
 '1490.jpg',
 '1493.jpg',
 '1597.jpg',
 '1648.jpg',
 '289.jpg',
 '3005.jpg',
 '3008.jpg',
 '3071.jpg',
 '3072.jpg',
 '3101.jpg',
 '311.jpg',
 '3149.jpg',
 '3180.jpg',
 '325.jpg',
 '326.jpg',
 '3378.jpg',
 '3391.jpg',
 '340.jpg',
 '3400.jpg',
 '3401.jpg',
 '3402.jpg',
 '3415.jpg',
 '3416.jpg',
 '3428.jpg',
 '345.jpg',
 '3488.jpg',
 '3489.jpg',
 '3490.jpg',
 '3560.jpg',
 '3589.jpg',
 '3692.jpg',
 '3693.jpg',
 '3694.jpg',
 '480.jpg',
 '549.jpg',
 '568.jpg',
 '621.jpg',
 '676.jpg',
 '688.jpg',
 '689.jpg',
 '714.jpg',
 '722.jpg',
 '779.jpg',
 '780

In [73]:
len(idx_lst)

98

In [74]:
error_lst=[]
result_lst=[]
pass_lst=[]
for idx in idx_lst:
    #print(idx)
    img=Image.open(path+idx)
    result=pytesseract.image_to_string(img,lang='kor')

    result=result.replace(' ','')
    a=re.sub('[\n]','',result)
    #print(a)
    
    try:
        # 원료로 시작하는 부분 찾기
        start_idx=re.search('원료',a).start()

        # 끝나는 부분 찾기
        min_len=10000
        min_idx=0
        word_idx=0
        word_lst=['중량','보관','반품','포장','내용','폴리','나용','본제품','유전자','용황','품목',
                  '영양','내리','유통','포징','포함','제조원','면리에틸','구입','식품','소비'] # 계속 추가하기
        while 1:
            # break문
            if word_idx>len(word_lst):
                break
            
            # 최소글자수 찾기
            try:
                end_idx=re.search(word_lst[word_idx],a).start()
                if end_idx<start_idx:
                    word_len=len(a[start_idx:])
                else:
                    word_len=len(a[start_idx:end_idx])

                if word_len<min_len:
                    min_len=word_len
                    min_idx=word_idx
                
                word_idx+=1

            except:
                word_idx+=1

        end_idx=re.search(word_lst[min_idx],a).start()
        result_lst.append(a[start_idx:end_idx])
        #print(a[start_idx:end_idx],'\n')

        pass_lst.append(idx)

    except:
        error_lst.append(idx)
        #print('error\n')

In [75]:
print('pass',len(pass_lst))
print('result',len(result_lst))
print('error',len(error_lst))

pass 48
result 48
error 50


In [130]:
result_lst

['원료명:유기농단풍시렵(0090)',
 '원료명:유기농압착올리브유100%*',
 '원료및함량:대592%(',
 '원료명및함량:계피(100%)',
 '원료명:유기농압착아보카도오일100%*반웅장수:',
 '원료명:포도원액(무수아황샌(산화방지제))70%,와인식초(포도,메타중아황산칼륨)30%,',
 '원료명201국산)10%,곤드래나물(국산)10%,부지갱이나물(국산)10%,가지(아비본빠티비그거0,계란,콩기름,고추장,00180093092119',
 '원료명및함량:유기농옥수수(83.8%),정제수,정제소금',
 '원료명:브유100%＊',
 '원료및함량:가쓰오부시(4301:',
 '원료명:포도원액70%,와인식초29.95무수아황샌산화방지제)*',
 '원료명-박력분,리코타치즈(유청,우유,유크림,정제소금,구연산)229,계란,시금치1196,듀렁밀,정제수,브레드크럼(박력분,정제수,정제소금,효모),그레이티드치즈(우유,정제소금,렌넷,리소징(계란)),정제소금,육두구,파프리카/',
 '원료명:압착올리브유99.89%,천연바쌀향0.2%*',
 '원료명1이미티애나지락살(중국|토[메루애티0006000여|미다따다기해름.밀.토마토:징어.조개류(쿨,합찾5:',
 '',
 '원료명:발사믹식초90%|포도원액와인식초,무수아황산(신회탕지제|무화과추출몰!096(무화과(고형분100%)%처하과향1%한아라00000*',
 '원료명:유기농강낭콩(50%),유기농토마토쥬스(45%),유기농사탕수수설탕(2%),유기농식조(1%),정제소금,옥수수전분',
 '원료및함량:연유(우유).설탕,포도당시럽,버터10%,정제소금0.6%*',
 "원료명:'27266000800010301,타라겸,*",
 '원료명:우유유크림설탕코코너일초클릿칩유청타라검천연항료민트향|-',
 '원료명및함량:ㅁ-소르비톨98.43%,천연스피아민트향0.53%스테아린산칼슴.수크랄로스(감미료),스테비아,스피룰리나청색소,홍화황색소『',
 '원료명:"게몬에이드-장제수,설탕옥수수시렬.농축레몬주스말토믹스트린팩틴,구(\'게로에이드라즈케리-점제수설탕.지등.옥수수시럽레온주스말토덱스트린락스

In [76]:
stopword=['중국','대','국산','토','국내산','중국산','미국산','호주산','필리핀','인도네시아샌','자사제품','뤄미국호주','인도산','패중국산','프랑스 독일 등','외국산','프랑스']

In [92]:
ingredient_lst=[]
for result in result_lst:
    if len(result)==0:
        sen=''
    elif '원료및함량' in result:
        sen=result[5:]
    elif '원료명및함량' in result:
        sen=result[6:]
    elif '원료명및' in result:
        sen=result[4:]
    elif '원료명' in result:
        sen=result[3:]

    sen=re.sub('[()/*-|ㅣ:.{}%"!]',',',sen)
    sen=re.sub(',+',',',sen)
    sen=re.sub('[^가-힣,]','',sen)
    
    lst=[]
    if ',' in sen:
        for word in sen.split(','):
            word=word.strip()
            if len(word)>0:
                try:
                    spelled_sent=spell_checker.check(word)
                    spelled_sent=spelled_sent.as_dict()
                    word=spelled_sent['checked']
                    word=word.strip()
                except:
                    word=word.strip()

            if len(word)>0 and word not in stopword:
                lst.append(word)
    else:
        lst.append(sen.strip())
    ingredient_lst.append(lst)

In [93]:
len(ingredient_lst)

48

In [94]:
df1=pd.DataFrame({'img':pass_lst,'성분':ingredient_lst})
df1

,img,성분
0,1012.jpg,[유기농 단풍 시협]
1,1042.jpg,[유기농 압착 올리브유]
2,1057.jpg,[]
3,1073.jpg,[계피]
4,1089.jpg,"[유기농 압착 아보카도 오일, 반응 장수]"
5,1144.jpg,"[포도 원액, 무수아황산, 산화방지제, 와인식초, 포도, 메타 중 아황산 칼륨]"
6,115.jpg,"[곤드레 나물, 부지깽이나 물, 가지, 아비본빠티비그거, 계란, 콩기름, 고추장]"
7,1199.jpg,"[유기농 옥수수, 정제수, 정제소금]"
8,1209.jpg,[부유]
9,1273.jpg,[가쓰오부시]


In [139]:
lst1=df1['성분'].tolist()
lst1=sum(lst1, [])
lst1=list(set(lst1))
lst1

['',
 '육수 맛 조미 분',
 '후추분말마늘양념분말',
 '느니고촛가루',
 '건미역구운김',
 '조미쇠고기맛후레이크',
 '토참맛육수분말',
 '팡유',
 '건조꽃맞살어묵',
 '리소징',
 '무수 아황산',
 '책틴구아경구연산천연형류',
 '스테아린산칼슴',
 '미다따다기해름',
 '정제수',
 '분리대 두 단백 중국산',
 '혼합메밀분',
 '청양고추추출물분말고추',
 '달갤쇠고기함유',
 '수크랄로스',
 '합찾',
 '블러드 오렌지주스',
 '무화과추출몰',
 '곳가루',
 '진한감칠맛분',
 '트린',
 '터키샌',
 '혼합메밀가루',
 '버터난황포도당시럽',
 '장제수',
 '감칠맛조미본유지맛분말',
 '고추맛 베이스',
 '레몬쥬스농축',
 '탈지대두',
 '조미참깨볶음참깨조미참깨용조미액김가루조미김',
 '초잇가공파',
 '사',
 '애플 매정제',
 '스',
 '글 양념분말',
 '수프류',
 '맛비이스',
 '알긴산나트률럼주카라멜썩소천연향료크림향바닐라향커미향',
 '늘분참맛다대기양념분',
 '감칠맛베이스',
 '로스팅육수분말후추분말',
 '닭고기대두우유쇠고기함유',
 '메밀',
 '숙성 마늘맛 분',
 '진한감칠맛분후추분말',
 '계란',
 '적',
 '코리인데',
 '기타과당',
 '들까외국산',
 '스프류꼬도당정제소금',
 '레드칠리',
 '당',
 '매운 고추 양념분말',
 '옷이 둑 샌',
 '토마토',
 '여',
 '만능 간장소스',
 '두부',
 '포도 원액',
 '미국',
 '다진 마늘',
 '맛베이스',
 '아비본빠티비그거',
 '이스트 진액',
 '설탕 포도당 시럽 천연차 향료',
 '유기농 레몬주스',
 '국',
 '모로쿼삽',
 '현미',
 '칠리추출물',
 '칠리추출물산도조절제',
 '정제소금',
 '아주출기르',
 '분말',
 '글루텐정제소금면류첨가알칼리제',
 '계피',
 '효모추출물분말',
 '건고주입자밀',
 '가쓰오과립분',
 '메타 중 아황산 칼륨',
 '홍합',
 '옥수수',
 '탈찌환원우유',
 '들까

### 성분

In [140]:
path='./img/기준폴더/성분/'
idx_lst=os.listdir(path)
idx_lst

['1072.jpg',
 '1090.jpg',
 '1257.jpg',
 '1334.jpg',
 '1622.jpg',
 '3038.jpg',
 '3420.jpg',
 '3597.jpg']

In [141]:
len(idx_lst)

8

In [142]:
pass_lst=[]
result_lst=[]

for idx in idx_lst:
    print(idx)
    img=Image.open(path+idx)
    result=pytesseract.image_to_string(img,lang='kor')

    result=result.replace(' ','')
    a=re.sub('[\n]','',result)
    
    try:
        # 원료로 시작하는 부분 찾기
        start_idx=re.search('성분',a).start()

        # 끝나는 부분 찾기
        min_len=10000
        min_idx=0
        word_idx=0
        word_lst=['유통','제조'] # 계속 추가하기

        while 1:
            # break문
            if word_idx>len(word_lst):
                break
            
            # 최소글자수 찾기
            try:
                end_idx=re.search(word_lst[word_idx],a).start()
                if end_idx<start_idx:
                    word_len=len(a[start_idx:])
                else:
                    word_len=len(a[start_idx:end_idx])

                if word_len<min_len:
                    min_len=word_len
                    min_idx=word_idx
                
                word_idx+=1

            except:
                word_idx+=1

        end_idx=re.search(word_lst[min_idx],a).start()
        result_lst.append(a[start_idx:end_idx])
        print(a[start_idx:end_idx])

        pass_lst.append(idx)
        
    except:
        error_lst.append(idx)
        print('error')

1072.jpg
error
1090.jpg
error
1257.jpg
성분및함량:압착아보카도유100%
1334.jpg
error
1622.jpg
error
3038.jpg
성분명및함랭명인쌀조청95%[쌀98.7%(국내산),엿기름(국내산),효소]명인이만든호박조청5%[명인쌀조청92%,호박8%(국내산)]
3420.jpg
error
3597.jpg
error


In [143]:
print(len(pass_lst))
print(len(result_lst))

2
2


In [144]:
stopword=['국내산']

In [145]:
ingredient_lst=[]
for result in result_lst:
    result=result.replace('성분및함량:','')
    result=result.replace('성분명및함랭','')

    result=re.sub('[\(\)\[\]]',',',result)
    result=re.sub(',+',',',result)
    result=re.sub('[^가-힣,]','',result)

    spelled_sent=spell_checker.check(result)
    spelled_sent=spelled_sent.as_dict()
    result=spelled_sent['checked']
    
    lst=[]
    if ',' in result:
        for word in result.split(','):
            word=word.strip()
            if len(word)>0 and word not in stopword:
                lst.append(word.strip())
    else:
        lst.append(result.strip())
    ingredient_lst.append(lst)

In [146]:
ingredient_lst

[['압착아보카도유'], ['명인 쌀 조청', '쌀', '엿기름', '효소', '명인이 만든 호박 조청', '명인 쌀 조청', '호박']]

In [147]:
df2=pd.DataFrame({'img':pass_lst,'성분':ingredient_lst})
df2

,img,성분
0,1257.jpg,[압착아보카도유]
1,3038.jpg,"[명인 쌀 조청, 쌀, 엿기름, 효소, 명인이 만든 호박 조청, 명인 쌀 조청, 호박]"


In [148]:
lst2=df2['성분'].tolist()
lst2=sum(lst2,[])
lst2=list(set(lst2))
lst2

['엿기름', '호박', '명인이 만든 호박 조청', '쌀', '압착아보카도유', '명인 쌀 조청', '효소']

### 원재료-가로

In [149]:
path='./img/기준폴더/원재료-가로/'
idx_lst=os.listdir(path)
idx_lst

['10.jpg',
 '100.jpg',
 '1002.jpg',
 '1003.jpg',
 '1004.jpg',
 '1006.jpg',
 '1007.jpg',
 '1008.jpg',
 '1009.jpg',
 '101.jpg',
 '1011.jpg',
 '1015.jpg',
 '1016.jpg',
 '1017.jpg',
 '1019.jpg',
 '1020.jpg',
 '1021.jpg',
 '1023.jpg',
 '1026.jpg',
 '1028.jpg',
 '1031.jpg',
 '1032.jpg',
 '1033.jpg',
 '1035.jpg',
 '1036.jpg',
 '1038.jpg',
 '104.jpg',
 '1044.jpg',
 '1046.jpg',
 '105.jpg',
 '1051.jpg',
 '1053.jpg',
 '1054.jpg',
 '1055.jpg',
 '1056.jpg',
 '1058.jpg',
 '1059.jpg',
 '106.jpg',
 '1060.jpg',
 '1061.jpg',
 '1062.jpg',
 '1063.jpg',
 '1066.jpg',
 '1067.jpg',
 '1069.jpg',
 '107.jpg',
 '1070.jpg',
 '1071.jpg',
 '1075.jpg',
 '1077.jpg',
 '108.jpg',
 '1080.jpg',
 '1083.jpg',
 '1084.jpg',
 '1086.jpg',
 '1088.jpg',
 '109.jpg',
 '1091.jpg',
 '1093.jpg',
 '1094.jpg',
 '1095.jpg',
 '1097.jpg',
 '1099.jpg',
 '11.jpg',
 '110.jpg',
 '1100.jpg',
 '1101.jpg',
 '1102.jpg',
 '1103.jpg',
 '1104.jpg',
 '1106.jpg',
 '1107.jpg',
 '111.jpg',
 '1113.jpg',
 '112.jpg',
 '1120.jpg',
 '1125.jpg',
 '1126.jpg',
 

In [150]:
len(idx_lst)

1701

In [151]:
pass_lst=[]
result_lst=[]
i=0
for idx in idx_lst:
    img=Image.open(path+idx)
    result=pytesseract.image_to_string(img,lang='kor')

    result=result.replace(' ','')
    a=re.sub('[\n]','',result)
    
    try:
        # 원료로 시작하는 부분 찾기
        start_idx=re.search('원재료',a).start()
        # 끝나는 부분 찾기
        min_len=100000
        min_idx=0
        word_idx=0
        word_lst=['업소','이제','유통','에이티','포장','이제초자','아트','유기농인증','주의','알레르기',
                  '용기','보관','품목','반품','수입','함유','제조','내용','포징','식품',
                  '구입','찾을','원산지','뜨거운','보와','을사용','원산지','본제품','벽기스','포함',
                  '이왕신','을씬','뚜검','도비','장시간','모장','원료','포항','특성','품록',
                  '제품','소임원','사워','이한','정보','이는','일영양','경기도','보고번호','개봉',
                  '교객','이이','판매','를사용','티더','기타'] # 계속 추가하기

        while 1:
            # break문
            if word_idx==len(word_lst):
                break
            
            # 최소글자수 찾기
            try:
                end_idx=re.search(word_lst[word_idx],a).start()
                if end_idx<start_idx:
                    word_len=len(a[start_idx:])
                else:
                    word_len=len(a[start_idx:end_idx])

                if word_len<=min_len:
                    min_len=word_len
                    min_idx=word_idx
                
                word_idx+=1

            except:
                word_idx+=1

        if word_idx>len(word_lst)+1:
            result_lst.append(a[start_idx:])

        else:
            end_idx=re.search(word_lst[min_idx],a).start()
            result_lst.append(a[start_idx:end_idx])
        
        pass_lst.append(idx)
        
    except:
        error_lst.append(idx)
    i+=1
    if i%100==0:
        print(i)

In [152]:
print(len(pass_lst))
print(len(result_lst))

750
750


In [225]:
# 단어별 따로
stopword=['국내산','중국산','국산','호주','미국','독일','인도산',
          '외국산이헨티나미국브라질동','호주산','미국산','스페인','덴마크',
          '뉴질랜드산','말레이시아','베트남산','리시 이산','러시아산','태국산',
          '미국 호주','스페인','중국','칠레 남아공','제주산','태국','베트남',
          '인도네시아','이탈리아산','캐나다','뉴질랜드',
          '네 덜란','아일랜드','러시아','말레이시아산','이시','드',
          '가입','열도라도 화이트','국 샌','함량',
          '열량','에콰도르산','원산지','인도네 시 야산',
          '브라질','이탈리아','캐나다산','필리핀','아르헨티나',
          '벨기에','미얀마','외고산','미얀마산','파라과이']

# 다른 단어랑 같이
remove_wordlst=['외국산스페인','및함량','미국산','외국산','호주산','독일산',
                '말레이시아산','일부사용','중국산','등','남아공','스페인',
                '및','미국','독일','국산','네덜란드','우크라이나','말레이시아',
                '함량','용량','국내선','외국','기준이므로 개인의 갈','캐나다산',
                '중국','캐나다','국내산','러시아','식품첨가물','으나가','소비기한 글',
                '뭐국내샌양때','그 내산']

# 아예 제거
remove_lst=['제조사','수입','혼입','경기도','제공량','냉동','제조','영하','퍼지고',
            '소수지고','보살이','번호']

In [229]:
ingredient_lst=[]
i=0
for result in result_lst:
    # 원래죠 제거
    result=result.replace('원재료 및 햠량','')
    result=result.replace('원재료명및함량:','')
    result=result.replace('원재료및함량','')
    result=result.replace('원재료명','')
    result=result.replace('원재료영및함량','')
    result=result.replace('원재료','')

    # 글자만 살리기
    result=re.sub('[\(\)\[\]]',',',result)
    result=re.sub(',+',',',result)
    result=re.sub('[^가-힣,]','',result)

    lst=[]
    if ',' in result:
        for word in result.split(','):
            word=word.strip()
            
            # 맞춤법 체크 (단어로 따로 있는 경우)
            if len(word)>0:
                try:
                    spelled_sent=spell_checker.check(word)
                    spelled_sent=spelled_sent.as_dict()
                    word=spelled_sent['checked']
                    word=word.strip()
                except:
                    word=word.strip()

            # 필요없는 말 제거(붙어있는 경우)
            for remove_word in remove_wordlst:
                word=word.replace(remove_word,'')

            # 아예 제거
            for remove_word in remove_lst:
                if remove_word in word:
                    word=''

            # 단어 제거
            word=word.strip()
            if (len(word)>0) and (word not in stopword) and len(word)<15:
                #print(word)
                lst.append(word)
    else:
        lst.append(result.strip())
    ingredient_lst.append(lst)
    i+=1
    if i%50==0:
        print(i)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750


In [230]:
print(len(ingredient_lst))
print(ingredient_lst[:2])

750
[['정제수', '마늘햄 시즈닝', '핫 스파이스 베이스 글'], ['베이컨', '돼지고기', '삼겹살', '전일영', '설탕', '혼합제제', '젖산 나트륨', '이 초산나트륨', '혼합제제 젖산 나트륨', '초산나트륨', '이산화규소', '터움바 소스', '정제수', '설탕', '조미식품 정제염', '쇠고기 양념분말', '글루탐 산나 트진 제', '설탕', '농죽간장카레소스', '양송이버섯 올리브유', '다진 마늘', '산 치즈가루', '치킨 파우더 우유 때도', '돼지고기', '알류', '쇠고기', '닭고기', '토마토', '조개류', '굴']]


In [231]:
df3=pd.DataFrame({'img':pass_lst,'성분':ingredient_lst})
df3

,img,성분
0,10.jpg,"[정제수, 마늘햄 시즈닝, 핫 스파이스 베이스 글]"
1,100.jpg,"[베이컨, 돼지고기, 삼겹살, 전일영, 설탕, 혼합제제, 젖산 나트륨, 이 초산나트..."
2,1002.jpg,"[감자 전분, 찹쌀]"
3,1008.jpg,"[맞절, 해바라기유, 치즈, 우유, 양유, 정제소금, 렌고, 캐세요 벗, 정제소금,..."
4,1016.jpg,[포도씨유]
...,...,...
745,969.jpg,"[화이트와 인식도, 백포도 수, 부수 아황산, 산화방지제, 포도농죽액배합포도, 알레르기]"
746,97.jpg,"[장, 다 뒤, 이호준, 밀이, 천일 엠]"
747,970.jpg,"[제사믹메달, 핑크, 년숙성카바버내미]"
748,984.jpg,"[밀가루, 설탕, 변성전분, 히드록시프로필전분초산전분, 베이킹파우더, 제일인산칼슴,..."


In [235]:
print(df1.shape,df2.shape,df3.shape)
df=pd.concat([df1,df2,df3])
print(df.shape)
df.head()

(48, 2) (2, 2) (750, 2)
(800, 2)


,img,성분
0,1012.jpg,[유기농 단풍시럽]
1,1042.jpg,[유기농 압착 올리브유]
2,1057.jpg,[]
3,1073.jpg,[계피]
4,1089.jpg,"[유기농 압착 아보카도 오일, 반응 장수]"


In [236]:
df.to_csv('./data/img_ingredient_1.csv',encoding='utf-8-sig',index=False)

In [240]:
lst=df['성분'].tolist()
lst=sum(lst,[])
lst=list(set(lst))
lst=lst[1:]
lst

['새우분맡글루탐산나트률',
 '올리고 당가 공품',
 '옥수수전분 농축 레몬주스',
 '느니고촛가루',
 '카스텔라',
 '구연산철',
 '건미역구운김',
 '동기릉',
 '팡유',
 '국사무',
 '참깨',
 '셀러리',
 '합성팽칭제',
 '한전해움융음소금나트륨',
 '만가닥 버섯',
 '책틴구아경구연산천연형류',
 '기타 과당 혼합분유',
 '유기농블랙커런트농축즘액',
 '퓨어메이플시우유대두밀계란',
 '탄산수 소나 트림',
 '이산화탄소',
 '볶은 소금',
 '아보카도 오일',
 '나 마요네즈',
 '쇼트닝 콜롬비아산',
 '미다따다기해름',
 '레스턴',
 '우유 메밀',
 '토코 피롤',
 '분리대 두 단백 중국산',
 '리얼 초코칩',
 '자복 고구마',
 '이산화 티타늄',
 '이베리아 햄이 플레 버드',
 '착색료뻐카로틴',
 '농후 발효유인 유',
 '휘핑크림',
 '파미봇도타우유',
 '크링',
 '마누라 분말',
 '사과착즘액',
 '쑥갓',
 '농축 레몬주스',
 '농축유청',
 '달갤쇠고기함유',
 '강자',
 '난황액난황',
 '명태 연육',
 '든 쌀',
 '이스트',
 '무화과추출몰',
 '마다이서어으곤천연향료',
 '볶음 대두',
 '팡핵유',
 '통발 앙금',
 '노르웨이산',
 '밀글루텐',
 '자연 키즈모 말릴라 치즈',
 '탁주',
 '코코아 분말 천일염 밀가루',
 '고추맛 베이스',
 '새우게',
 '탄산수 소나 트롤',
 '향미증진제 양파',
 '분말 셀룰로스',
 '에스테르화유',
 '톨시럽',
 '치킨향',
 '탈지대두',
 '조미참깨볶음참깨조미참깨용조미액김가루조미김',
 '급 유기농 원유',
 '방일 부용',
 '사',
 '산 니트',
 '전립분',
 '카카오',
 '거사분',
 '고초 가루 인상',
 '고조가루혼함간장촉설탕',
 '세르비아  혼합간장',
 '량이',
 '청경채',
 '초산나트률',
 '쌀 옥수수 감자',
 '황태머리',
 '교촌 프리미엄 전용 유',
 '제재소',
 '보일드 치킨',
 '다

In [241]:
with open('./list/img_ingredient_1.txt','w',encoding='UTF-8') as f:
    for word in lst:
        f.write(word+'\n')

## 2000번대 데이터

### 원재료-가로

In [3]:
path='./img/기준폴더/2000/원재료-가로/'
idx_lst=os.listdir(path)
idx_lst

['2002.jpg',
 '2003.jpg',
 '2008.jpg',
 '2009.jpg',
 '2010.jpg',
 '2011.jpg',
 '2012.jpg',
 '2014.jpg',
 '2015.jpg',
 '2016.jpg',
 '2017.jpg',
 '2018.jpg',
 '2019.jpg',
 '2020.jpg',
 '2021.jpg',
 '2023.jpg',
 '2025.jpg',
 '2026.jpg',
 '2027.jpg',
 '2028.jpg',
 '2030.jpg',
 '2031.jpg',
 '2033.jpg',
 '2035.jpg',
 '2036.jpg',
 '2037.jpg',
 '2038.jpg',
 '2041.jpg',
 '2042.jpg',
 '2043.jpg',
 '2044.jpg',
 '2046.jpg',
 '2048.jpg',
 '2049.jpg',
 '2050.jpg',
 '2051.jpg',
 '2052.jpg',
 '2053.jpg',
 '2054.jpg',
 '2055.jpg',
 '2056.jpg',
 '2058.jpg',
 '2059.jpg',
 '2060.jpg',
 '2061.jpg',
 '2062.jpg',
 '2063.jpg',
 '2064.jpg',
 '2065.jpg',
 '2066.jpg',
 '2068.jpg',
 '2070.jpg',
 '2071.jpg',
 '2072.jpg',
 '2074.jpg',
 '2075.jpg',
 '2076.jpg',
 '2077.jpg',
 '2078.jpg',
 '2079.jpg',
 '2080.jpg',
 '2082.jpg',
 '2087.jpg',
 '2089.jpg',
 '2090.jpg',
 '2094.jpg',
 '2095.jpg',
 '2098.jpg',
 '2099.jpg',
 '2101.jpg',
 '2103.jpg',
 '2104.jpg',
 '2105.jpg',
 '2106.jpg',
 '2108.jpg',
 '2109.jpg',
 '2111.jpg',

In [11]:
len(idx_lst)

566

In [5]:
pass_lst=[]
result_lst=[]
error_lst=[]
i=0
for idx in idx_lst:
    img=Image.open(path+idx)
    result=pytesseract.image_to_string(img,lang='kor')

    result=result.replace(' ','')
    a=re.sub('[\n]','',result)
    
    try:
        # 원료로 시작하는 부분 찾기
        start_idx=re.search('원재료',a).start()
        # 끝나는 부분 찾기
        min_len=100000
        min_idx=0
        word_idx=0
        word_lst=['업소','이제','유통','에이티','포장','이제초자','아트','유기농인증','주의','알레르기',
                  '용기','보관','품목','반품','수입','함유','제조','내용','포징','식품',
                  '구입','찾을','원산지','뜨거운','보와','을사용','원산지','본제품','벽기스','포함',
                  '이왕신','을씬','뚜검','도비','장시간','모장','원료','포항','특성','품록',
                  '제품','소임원','사워','이한','정보','이는','일영양','경기도','보고번호','개봉',
                  '교객','이이','판매','를사용','티더','기타'] # 계속 추가하기

        while 1:
            # break문
            if word_idx==len(word_lst):
                break
            
            # 최소글자수 찾기
            try:
                end_idx=re.search(word_lst[word_idx],a).start()
                if end_idx<start_idx:
                    word_len=len(a[start_idx:])
                else:
                    word_len=len(a[start_idx:end_idx])

                if word_len<=min_len:
                    min_len=word_len
                    min_idx=word_idx
                
                word_idx+=1

            except:
                word_idx+=1

        if word_idx>len(word_lst):
            result_lst.append(a[start_idx:])

        else:
            end_idx=re.search(word_lst[min_idx],a).start()
            result_lst.append(a[start_idx:end_idx])
        
        pass_lst.append(idx)
        
    except:
        error_lst.append(idx)
    i+=1
    if i%100==0:
        print(i)

100
200
300
400
500


In [6]:
print(len(pass_lst))
print(len(result_lst))
print(len(error_lst))

287
287
279


In [18]:
result_lst

['원재료명아라비카원두100%(과테말라산)고',
 '원재료명:볶은커피원두100%(미국산)',
 '원재료명뽀}커피원두100%ㅣ|',
 '원재료명커피원두10146커피|',
 '원재료명및함량|커피원두100%(',
 '원재료명및함량커피추출액고형분3.0%이상/생두에티오피아산100%)700%',
 '원재료및함량|인도30%,브라질20%,에티오피아15%,베트남5%)땅차커피/인천광역시서구000',
 '원재료및함량ㅣ커피원두1009%(에티오피아309코스타리카409니카라과309)ㆍ',
 '원재료명가콕산],흔함제제(구연산수산화칼륨)},설탕우유농축분말,(독일산),커피(콜롬비아산40%,케나산:30%),포밍크리머(|콘함설할류바닐라문망)정제솜,바닐라추충문말제상0-티거미바닐라라떼가를잔에넣고적당향60700의',
 '원재료명및함량|커피원두100%글라시코블렌드(브라질산50%콜롬비아산50%)삼페인블렌드(에티오피아산50%케나산50%)엘살바도르(엘살바도르산100%)에티오피아(에티오피아산100%)',
 '원재료명및함량커피원두100%',
 '',
 '원재료명및함량커피원두100%포장재질내포장/폴리프로필렌',
 "원재료명서피저6650%,에티오피아2086,베트남20%콜콜롬비아10%'포장재료|폴리에틸렌(66)품목보고번호|2017036843792ㆍ",
 '원재료명및함량|원두커피100%',
 '원재료및함량ㅡ원두거피100(브라질5096,콜움비아2596,과테말라2596)포장재절ㅡ내피/알루미늄,외피/종이',
 '원재료명[커피원두100\\',
 '원재료명커피(에티오피아)100%',
 '원재료및함량|커피원두100%(',
 '원재료명및함량|커피원두100%외면:프로테고(종이)',
 '원재료및함량|커피원두100%(',
 '원재료명및함량커피원두100%(브라질85%.,에티오피아15%)00006제조원(주)케이코닉',
 '원재료명및함량[커피',
 '원재료명:원두커피100%-',
 '원재료명드산).4<1오일(코코너일필리핀산)유화저].디카페인커피추출액(브라질|년0을가기세이:탄산칼륨}탄커피디',
 '원재료명및함량|커피주졸역100%(고형분3.4

In [7]:
# 단어별 따로
stopword=['중국산','과테말라산','고','미국산','인도','브라질','에티오피아','베트남','독일산',
          '콜롬비아산','및 함량','에티오피아산','케냐산','브라질','콜움비아','과테말라','제조원',
          '인도산','외','중량','케냐','인도네시아','영국','태국','이집트','나이지리아','미국',
          '인도','독일','국산','제품명','칠레산','스리랑카산','아르헨티나산','국내산','베트남산',
          '뉴질랜드산','스페인','영양','캐나다','말레이시아산','아르헨티','아르헨티나','싱가포르산',
          '이탈리아산','벨기에산','성분','월','말레이시아','터키']

# 다른 단어랑 같이
remove_wordlst=['중국산','또','및함량','이상','에티오피산','인천광역시','에티오피아','및 함량',
                '브라질산','함량','스페인','및','항량','용량','제주산','공유 시','국산',
                '뉴질랜드 에','미국','인도네시아','영국이','캐나다산','이탈리안','미국산',
                '국내산','영양','프랑스','뉴질랜드산','등','호주산','함유','뒤','외',
                '아르헨티나산','말레이시아산','에서','독일','국내','험유','베트남산','삐트남산',
                '말레이사 야산','캐니다샌','뉴질랜드','항 유','샌']

# 아예 제거
remove_lst=['중국산','넣고','포장','품목','종이','년','인천','서울','제품명','품목번호','번호',
            '연구','정보','지방 일부','표기','열량','놓고','공장','총 내용','개입','식품유형',
            '같은','보관','반품','교환','방법','냉장','식품','국번','제품은']

In [8]:
ingredient_lst=[]
i=0
for result in result_lst:
    # 원재료 제거
    result=result.replace('원재료명및함량:','')
    result=result.replace('원재료및함량','')
    result=result.replace('원재료명','')
    result=result.replace('원재료영및함량','')
    result=result.replace('원재료및','')
    result=result.replace('원재료','')

    # 글자만 살리기
    result=re.sub('[\(\)\[\]:|%]',',',result)
    result=re.sub(',+',',',result)
    result=re.sub('[^가-힣,]','',result)
    
    lst=[]
    if ',' in result:
        for word in result.split(','):
            word=word.strip()
            
            # 맞춤법 체크
            if len(word)>0:
                try:
                    spelled_sent=spell_checker.check(word)
                    spelled_sent=spelled_sent.as_dict()
                    word=spelled_sent['checked']
                    word=word.strip()
                except:
                    word=word.strip()
            
            # 필요없는 말 제거(붙어있는 경우)
            for remove_word in remove_wordlst:
                word=word.replace(remove_word,'')

            # 아예 제거
            for remove_word in remove_lst:
                if remove_word in word:
                    word=''

            # 단어 제거(따로 있는 경우)
            word=word.strip()
            if (len(word)>0) and (word not in stopword) and len(word)<15:
                lst.append(word)
                #print(word)
    else:
        lst.append(result.strip())
    ingredient_lst.append(lst)
    i+=1
    if i%50==0:
        print(i)

50
100
150
200
250


In [9]:
print(len(ingredient_lst))
print(len(pass_lst))

287
287


In [10]:
df4=pd.DataFrame({'img':pass_lst,'성분':ingredient_lst})
df4

,img,성분
0,2002.jpg,[아라비카 원두]
1,2008.jpg,"[볶은 커피 원두, 미]"
2,2010.jpg,[커피 원두]
3,2011.jpg,[커피 원두커피]
4,2012.jpg,[커피 원두]
...,...,...
282,2979.jpg,"[크랜베리 쿠키, 크랜베리]"
283,2991.jpg,"[감자, 해바라기씨유, 겸은 서양 송로 시즈닝, 감자말토덱스트린, 정제수, 금, 설..."
284,2996.jpg,"[버터, 이 달라야, 문항, 유기농 밀가루 설탕]"
285,2998.jpg,"[유기농 찹쌀, 약밥콕견과, 무농약생멍콩, 무농약 건밤, 무농약 대추, 무능악깐호두..."


In [12]:
df4.to_csv('./data/img_ingredient_2.csv',encoding='utf-8-sig',index=False)

In [14]:
lst=df4['성분'].tolist()
lst=sum(lst,[])
lst=list(set(lst))
lst=lst[1:]
lst

['천일염',
 '대두 고형분',
 '아몬드',
 '메이플 시럽',
 '소르비이트대두레시딘',
 '도유흐모유당감자전분대두분말',
 '정제수정 져 소',
 '일반증류주',
 '알룰로스',
 '파마산 지즈',
 '밀크 파우더',
 '명동 이스트',
 '체르벨',
 '산 바나나',
 '설탕 천일염',
 '호모곡가공품소스',
 '질산 갈륨',
 '천연향료',
 '탄산 수소 나트를',
 '파블로 네루다',
 '말이 이시아 산유 화제',
 '필링 계란',
 '아르 렌티나',
 '당류가공품 당류가공품',
 '및함량투딜리셔스씨케이성남지점',
 '기털탱벌탕전분',
 '코코아리퀘코코이매스',
 '유기농 브라질',
 '잉글리시 브렉퍼스트',
 '통발 앙금',
 '박력분',
 '포함',
 '천연 레몬향',
 '폴리에틸렌',
 '코코아 매스',
 '재스민 꽃',
 '아메스',
 '해이즐덧',
 '체리',
 '신도 조절제 산도 조절제',
 '유화 저',
 '폴리',
 '견과류 가공품',
 '언앙누슴쟁응설창탄수',
 '미 설탕 전란 역',
 '가공이 크림',
 '로커스트콩검잔탄검',
 '심황색소프랄린아문드',
 '입',
 '전동육포염지제정백당정제소금',
 '전지분유유정분말우유단백질',
 '빙초산',
 '천연향료 딸기향',
 '글라스 코 블렌드',
 '캐러멜 시럽',
 '올리고당',
 '천연 딸기향',
 '치즈 컬처',
 '헤이즐 벗',
 '체더치즈',
 '가공버터뉴질련드써오일우위',
 '분대',
 '여 소 말 트',
 '산알류',
 '중국 말린 사과 조각',
 '난 백약 마가린',
 '핵이',
 '미곡',
 '젖산람슴',
 '밀감류 말',
 '나트 옴',
 '마가린퓨',
 '맵쓸',
 '정제팡유',
 '전지분유대두려시틴',
 '올리브',
 '글루탐산나트륨',
 '설탕 전란',
 '산',
 '웨이퍼',
 '엘란',
 '동팔앙금통팔용삶음팔',
 '유기농 딸기 주스',
 '슬루물용',
 '가공버터',
 '전란액설탕밀기리밀미',
 '앤디엄',
 '유토 한',
 '조제해수염화마그네슴혼합제제',


In [15]:
with open('./list/img_ingredient_2.txt','w',encoding='UTF-8') as f:
    for word in lst:
        f.write(word+'\n')

## 규칙데이터 통합 + 1차 리스트 -> 2차 리스트업

In [8]:
df1=pd.read_csv('./data/img_ingredient_1.csv',encoding='utf-8')
df2=pd.read_csv('./data/img_ingredient_2.csv',encoding='utf-8')
df3=pd.read_csv('./data/img_ingredient_2000_final.csv',encoding='utf-8')
df=pd.concat([df1,df2,df3])
df.reset_index(drop=True,inplace=True)
print(df.shape)
df

(1103, 2)


,img,성분
0,1012.jpg,['유기농 단풍시럽']
1,1042.jpg,['유기농 압착 올리브유']
2,1057.jpg,[]
3,1073.jpg,['계피']
4,1089.jpg,"['유기농 압착 아보카도 오일', '반응 장수']"
...,...,...
1098,2385.jpg,"['연성가공치즈', '자연치즈', '탈지유', '슈크림', '버터밀크파우뎌', '딥..."
1099,2386.jpg,"['계란', '초콜릿', '코코아', '코코아 버터', '설탕', '레시틴', '바..."
1100,2387.jpg,"['계란', '식물성 크림', '정제수', '액상과당', '경화팡핵유', '레 세틴..."
1101,2388.jpg,"['계란', '식물성 크림', '정제수', '액상과당', '경화유', '레 세틴',..."


In [9]:
# 규칙 리스트
lst=df['성분'].tolist()
ingredient_lst=[]
for sen in lst:
    sen=re.sub("['\[\]]",',',sen)
    for word in sen.split(','):
        word=word.strip()
        if len(word)>0 and len(word)<15:
            ingredient_lst.append(word)
ingredient_lst=list(set(ingredient_lst))
print(len(ingredient_lst))
ingredient_lst[:5]

4899


['울엿천연망고향분리유청단백질', '성 향료', '틀어할 로스', '영양강화밀가루 설탕', '캐러멜 라이즈드 헤이즐']

In [10]:
# 1차 리스트 불러오기
with open("./list/total.txt", "r") as file:
    data = file.readlines()

for word in data:
    word=re.sub('\n','',word)
    ingredient_lst.append(word)
print(len(ingredient_lst))
ingredient_lst[:5]

16564


['울엿천연망고향분리유청단백질', '성 향료', '틀어할 로스', '영양강화밀가루 설탕', '캐러멜 라이즈드 헤이즐']

In [11]:
# 2차 리스트 완성
stopword=['러시아','벨기에','단백질','지방','콜레스테롤','미국','제품명','탄수화물','포화지방','개입','프랑스산','트랜스지방','살균제품',
          '인도네시아산','나트륨','영양정보','헝가리','시험','인도','이탈리아','유동 기한','소스','내면','국내','나 트림','제품 별도 표기일까지',
          '함유','캐나다산','포장','냉동','칠레산','캐다','말레이시아산','트련스지방','필란드','덴마크','탄산나트륨','라 님','변성 전문',
          '러시 이산','필리핀','품목','다 갈','공정위 고시','베트남산','중국','이태리','폴리에틸렌','식품','엑시코산','중량','멕시코',
          '알레르기','유통 전문','수화물','국신','대만산','영문','파키스탄산','포함','종국','곡산','향유','나트','국선','무농약','나이지리아',
          '영양','내량','동호']

lst=[]
for word in ingredient_lst:
    if word not in stopword:
        lst.append(word)

In [12]:
print(len(lst))
lst[:5]

16493


['울엿천연망고향분리유청단백질', '성 향료', '틀어할 로스', '영양강화밀가루 설탕', '캐러멜 라이즈드 헤이즐']

## 전체 이미지 -> 성분 텍스트

In [13]:
path='./img/img/'
idx_lst=os.listdir(path)
idx_lst

['0.jpg',
 '1.jpg',
 '10.jpg',
 '100.jpg',
 '1000.jpg',
 '1001.jpg',
 '1002.jpg',
 '1003.jpg',
 '1004.jpg',
 '1005.jpg',
 '1006.jpg',
 '1007.jpg',
 '1008.jpg',
 '1009.jpg',
 '101.jpg',
 '1010.jpg',
 '1011.jpg',
 '1012.jpg',
 '1013.jpg',
 '1014.jpg',
 '1015.jpg',
 '1016.jpg',
 '1017.jpg',
 '1018.jpg',
 '1019.jpg',
 '102.jpg',
 '1020.jpg',
 '1021.jpg',
 '1022.jpg',
 '1023.jpg',
 '1024.jpg',
 '1025.jpg',
 '1026.jpg',
 '1027.jpg',
 '1028.jpg',
 '1029.jpg',
 '103.jpg',
 '1030.jpg',
 '1031.jpg',
 '1032.jpg',
 '1033.jpg',
 '1034.jpg',
 '1035.jpg',
 '1036.jpg',
 '1037.jpg',
 '1038.jpg',
 '1039.jpg',
 '104.jpg',
 '1040.jpg',
 '1041.jpg',
 '1042.jpg',
 '1043.jpg',
 '1044.jpg',
 '1045.jpg',
 '1046.jpg',
 '1047.jpg',
 '1048.jpg',
 '1049.jpg',
 '105.jpg',
 '1050.jpg',
 '1051.jpg',
 '1052.jpg',
 '1053.jpg',
 '1054.jpg',
 '1055.jpg',
 '1056.jpg',
 '1057.jpg',
 '1058.jpg',
 '1059.jpg',
 '106.jpg',
 '1060.jpg',
 '1061.jpg',
 '1062.jpg',
 '1063.jpg',
 '1064.jpg',
 '1065.jpg',
 '1066.jpg',
 '1067.jpg',
 

In [14]:
len(idx_lst)

3756

In [15]:
result_lst=[]
i=0
for idx in idx_lst:
    # 텍스트 추출
    img=Image.open(path+idx)
    result=pytesseract.image_to_string(img,lang='kor')

    # 특수문자제거
    result=re.sub('[\(\)\[\]/|:\nㆍ%\{\}.]',',',result)
    result=re.sub('[^가-힣,]','',result)
    result=re.sub(',+',',',result)

    idx_ingredient_lst=[]
    for word in result.split(','):
        word=word.strip()
        if len(word)>1:
            # 맞춤법 검사
            try:
                spelled_sent=spell_checker.check(word)
                spelled_sent=spelled_sent.as_dict()
                word=spelled_sent['checked']
            except:
                word=word.strip()

            # 성분리스트에 있는 거 저장
            if word in lst:
                idx_ingredient_lst.append(word)

    idx_ingredient_lst=list(set(idx_ingredient_lst))
    result_lst.append(idx_ingredient_lst)
    i+=1
    if i%10==0:
        print(i)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [16]:
print(len(idx_lst))
print(len(result_lst))

3756
3756


In [17]:
img_df=pd.DataFrame({'img':idx_lst,'이미지성분':result_lst})
img_df

,img,이미지성분
0,0.jpg,"[고등어, 아황산류, 메밀, 옥수수유 옥수수, 복숭아, 오징어, 홍합 포함, 설탕,..."
1,1.jpg,"[조개류, 정제소금, 주정, 떡류, 조기, 소스 소스, 참치, 비타민, 가공소금, ..."
2,10.jpg,"[돼지고기, 난백분, 산도 조절제, 탈지 대도, 돈 지방, 향미증진제, 리투아니아,..."
3,100.jpg,"[고등어, 호두, 올리브유, 팡핵유, 탄산칼슴, 아황산류, 땅콩, 산 치즈가루, 복..."
4,1000.jpg,"[적포도, 전복]"
...,...,...
3751,995.jpg,"[옥수수, 글리세린지방산에스테르, 현미, 유화제, 건면, 감자 전분, 당류]"
3752,996.jpg,"[정제소금, 과채가공품, 해바라기씨유]"
3753,997.jpg,[]
3754,998.jpg,[고춧가루]


In [18]:
img_df.to_csv('./data/kurly_ingredient.csv',encoding='utf-8-sig',index=False)